In [12]:
# I/O
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
from io import BytesIO
from zipfile import ZipFile
from datetime import datetime, timedelta

In [2]:
# URL of the zipped data file
urls = ['https://s3.amazonaws.com/capitalbikeshare-data/202401-capitalbikeshare-tripdata.zip',
        'https://s3.amazonaws.com/capitalbikeshare-data/202402-capitalbikeshare-tripdata.zip', 
       'https://s3.amazonaws.com/capitalbikeshare-data/202403-capitalbikeshare-tripdata.zip']

In [3]:
# List to store DataFrames
dataframes = []
# Loop through each URL
for url in urls:
    # Download the zipped file
    response = requests.get(url)
    zipfile = ZipFile(BytesIO(response.content))
    
    # List files in the zip file
    file_name = zipfile.namelist()[0]
    
    # Read the desired file into a pandas DataFrame
    with zipfile.open(file_name) as file:
        combined_df = pd.read_csv(file)
        dataframes.append(combined_df)

# Concatenate all DataFrames into one
combined_combined_df = pd.concat(dataframes, ignore_index=True)

In [4]:
# Display the DataFrame
combined_combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,748A93D7DE8A41CD,classic_bike,2024-01-25 15:49:59,2024-01-25 15:52:35,1st & O St NW,31519.0,1st & L St NW,31677.0,38.908643,-77.012365,38.903819,-77.011987,member
1,75CBFD136F06305B,classic_bike,2024-01-02 16:44:58,2024-01-02 16:53:25,1st & O St NW,31519.0,4th & College St NW,31138.0,38.908643,-77.012365,38.921233,-77.018135,member
2,0536C9720F87E04C,classic_bike,2024-01-24 15:40:15,2024-01-24 15:43:55,1st & O St NW,31519.0,1st & L St NW,31677.0,38.908643,-77.012365,38.903819,-77.011987,member
3,9E17390C218783B5,classic_bike,2024-01-04 15:35:00,2024-01-04 15:37:35,1st & O St NW,31519.0,1st & L St NW,31677.0,38.908643,-77.012365,38.903819,-77.011987,member
4,00727D0E773CDFF7,electric_bike,2024-01-05 12:27:58,2024-01-05 12:35:40,1st & O St NW,31519.0,10th & G St NW,31274.0,38.908690,-77.012317,38.898243,-77.026235,casual


# Exploratory Data Analysis  

Let's start creating new variables within our dataset and taking a look at its structure. 

In [5]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436946 entries, 0 to 436945
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             436946 non-null  object 
 1   rideable_type       436946 non-null  object 
 2   started_at          436946 non-null  object 
 3   ended_at            436946 non-null  object 
 4   start_station_name  371894 non-null  object 
 5   start_station_id    371894 non-null  float64
 6   end_station_name    368605 non-null  object 
 7   end_station_id      368500 non-null  float64
 8   start_lat           436946 non-null  float64
 9   start_lng           436946 non-null  float64
 10  end_lat             436558 non-null  float64
 11  end_lng             436558 non-null  float64
 12  member_casual       436946 non-null  object 
dtypes: float64(6), object(7)
memory usage: 43.3+ MB


Start creating variables: 
- Start and End Dates
- Start and End Times
- Convert Station IDs to Character Integers
- Create new geocombined_dfs that have start and end locations so that we can create the neighborhoods that they started and ended in
- 

In [6]:
combined_df["started_at"]= pd.to_datetime(combined_df["started_at"])
combined_df["started_at_date"]= pd.to_datetime(combined_df["started_at"]).dt.date
combined_df["started_at_time"]= pd.to_datetime(combined_df["started_at"]).dt.time
combined_df["ended_at"]= pd.to_datetime(combined_df["ended_at"])
combined_df["ended_at_date"]= pd.to_datetime(combined_df["ended_at"]).dt.date
combined_df["ended_at_time"]= pd.to_datetime(combined_df["ended_at"]).dt.time
combined_df["duration"] = pd.to_datetime(combined_df["ended_at"]) - pd.to_datetime(combined_df["started_at"])
combined_df["duration_minutes"] = combined_df.duration.apply(lambda td: td.total_seconds() / 60)
combined_df["start_station_id"] = combined_df["start_station_id"].astype(str)
combined_df["end_station_id"] = combined_df["end_station_id"].astype(str) 
# Filter dataframe to only have rides between 30 seconds and 2 hours
og_df_size = combined_df.shape[0]
combined_df = combined_df.loc[(combined_df['duration_minutes']>=.5) & (combined_df['duration_minutes']<=120) ]

In [7]:
og_df_size
new_df_size = combined_df.shape[0]
new_df_size/og_df_size

0.9777546882223432

In [8]:
combined_df.describe()

,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,duration,duration_minutes
count,427226,427226,427226.000000,427226.000000,427226.000000,427226.000000,427226,427226.000000
mean,2024-03-17 10:53:56.171096832,2024-03-17 11:08:53.587810816,38.904837,-77.031332,38.903965,-77.031151,0 days 00:14:57.416713870,14.956945
min,2024-03-01 00:00:22,2024-03-01 00:07:47,38.760000,-77.370000,38.710000,-77.440000,0 days 00:00:30,0.500000
25%,2024-03-10 15:43:06.249999872,2024-03-10 15:56:52,38.892223,-77.043074,38.890540,-77.043182,0 days 00:05:49,5.816667
50%,2024-03-17 12:01:29,2024-03-17 12:19:46,38.903750,-77.030000,38.903040,-77.030000,0 days 00:10:07,10.116667
75%,2024-03-24 18:41:54.249999872,2024-03-24 19:00:39.750000128,38.917668,-77.012344,38.915544,-77.012289,0 days 00:17:40,17.666667
max,2024-03-31 23:58:41,2024-04-01 00:50:05,39.125828,-76.820000,39.125828,-76.820000,0 days 02:00:00,120.000000
std,NaN,NaN,0.026159,0.034113,0.026088,0.034024,0 days 00:15:35.989089177,15.599818


Create two new dataframes: 
1. Start times combined_df that has the location they started at if it has a station
2. End times combined_df that has the location they ended at if it has a station 

In [9]:
start_stations = combined_df[["start_station_name","start_station_id","start_lat","start_lng"]]
end_stations = combined_df[["end_station_name","end_station_id","end_lat","end_lng"]]
start_stations = start_stations.loc[start_stations['start_station_name'].notna()]
end_stations = end_stations.loc[end_stations['end_station_name'].notna()]
start_stations.rename(columns = {"start_station_name":"station_name",
                                "start_station_id":"station_id",
                                "start_lat":"lat",
                                 "start_lng":"lng"}, inplace=True)

end_stations.rename(columns = {"end_station_name":"station_name",
                                "end_station_id":"station_id",
                                "end_lat":"lat",
                                 "end_lng":"lng"}, inplace=True )
stations = pd.concat([start_stations, end_stations])
stations = stations.drop_duplicates(subset=["station_name","station_id"])

In [10]:
combined_gdf = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.lng, stations.lat), crs="EPSG:4326")

In [11]:
combined_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 824 entries, 0 to 344888
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   station_name  824 non-null    object  
 1   station_id    824 non-null    object  
 2   lat           824 non-null    float64 
 3   lng           824 non-null    float64 
 4   geometry      824 non-null    geometry
dtypes: float64(2), geometry(1), object(2)
memory usage: 38.6+ KB


# Pull in Weather Data to add to rides

In [48]:
# Function to fetch weather data
def fetch_weather_data(lat, lon, date, api_key):
    url = "https://api.openweathermap.org/data/3.0/onecall/day_summary?"
    params = {
        'lat': lat,
        'lon': lon,
        'date': date.strftime('%Y-%m-%d'),
        'appid': api_key
    }
    response = requests.get(url, params=params)
    return response.json()
# Parameters
api_key = "32964f206a33b141c84d7f13e55a92d4"
dc_lat = 38.8950368
dc_lon = -77.0365427
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 3, 31)

# Create a list of dates for each day within the date range
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]


In [49]:
# Fetch data
weather_data = fetch_weather_data(dc_lat, dc_lon, start_date, api_key)


In [ ]:
weather_data

In [47]:
# Fetch data
daily_data = []
for date in date_range:
    weather_data = fetch_weather_data(latitude, longitude, date, api_key)
    daily_data.append({
        'date': weather_data['date'],
        'min_temp': weather_data['temperature']['min'],
        'max_temp': weather_data['temperature']['max'],
        'afternoon_temp': weather_data['temperature']['afternoon'],
        'night_temp': weather_data['temperature']['night'],
        'evening_temp': weather_data['temperature']['evening'],
        'morning_temp': weather_data['temperature']['morning'],
        'humidity': weather_data['humidity']['afternoon'],
        'pressure': weather_data['pressure']['afternoon'],
        'wind_speed': weather_data['wind']['max']['speed'],
        'wind_direction': weather_data['wind']['max']['direction'],
        'precipitation': weather_data['precipitation']['total'],
        'cloud_cover': weather_data['cloud_cover']['afternoon']
    })

NameError: name 'latitude' is not defined

In [ ]:
# Process and save data
weather_df = pd.DataFrame(daily_data)

# Start Analysis for Underserved Rides 

The idea for this analysis is that if 